In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv('C:/Users/82102/OneDrive - dongguk.edu/바탕 화면/2023.1학기 비어플/data/train.csv',encoding='utf-8')
test=pd.read_csv('C:/Users/82102/OneDrive - dongguk.edu/바탕 화면/2023.1학기 비어플/data/test.csv',encoding='utf-8')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132397 entries, 0 to 132396
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   132397 non-null  int64  
 1   CODE_GENDER          132397 non-null  object 
 2   CNT_CHILDREN         132397 non-null  object 
 3   AMT_INCOME_TOTAL     132397 non-null  float64
 4   NAME_EDUCATION_TYPE  132397 non-null  object 
 5   NAME_FAMILY_STATUS   132397 non-null  object 
 6   NAME_HOUSING_TYPE    132397 non-null  object 
 7   OCCUPATION_TYPE      132397 non-null  object 
 8   age                  132397 non-null  float64
 9   고용개월                 132397 non-null  float64
 10  MONTHS_BALANCE       132397 non-null  int64  
 11  STATUS               132397 non-null  object 
 12  begin                132397 non-null  int64  
 13  승인비율                 132397 non-null  float64
dtypes: float64(4), int64(3), object(7)
memory usage: 14.1+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56742 entries, 0 to 56741
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   56742 non-null  int64  
 1   CODE_GENDER          56742 non-null  object 
 2   CNT_CHILDREN         56742 non-null  object 
 3   AMT_INCOME_TOTAL     56742 non-null  float64
 4   NAME_EDUCATION_TYPE  56742 non-null  object 
 5   NAME_FAMILY_STATUS   56742 non-null  object 
 6   NAME_HOUSING_TYPE    56742 non-null  object 
 7   OCCUPATION_TYPE      56742 non-null  object 
 8   age                  56742 non-null  float64
 9   고용개월                 56742 non-null  float64
 10  MONTHS_BALANCE       56742 non-null  int64  
 11  STATUS               56742 non-null  object 
 12  begin                56742 non-null  int64  
 13  승인비율                 56742 non-null  float64
dtypes: float64(4), int64(3), object(7)
memory usage: 6.1+ MB


## ID 열은 학습에 사용하지 않으므로 삭제

In [5]:
train.drop(['ID'], axis=1,inplace=True)
test.drop(['ID'], axis=1,inplace=True)

## target 인 STATUS 승인은 1, 승인거부는 0으로 변환

In [6]:
train['STATUS'] = train['STATUS'].replace({'승인':1, '승인거부':0})
test['STATUS'] = test['STATUS'].replace({'승인':1, '승인거부':0})

## 범주형 변수는 One-hot Encoding, 수치형 변수는 Scaling

- train, test  각각 따로 진행하여 set 만듦

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [8]:
cate_train = train[['CODE_GENDER','CNT_CHILDREN','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']]
conti_train = train[['AMT_INCOME_TOTAL','age','고용개월','MONTHS_BALANCE','begin','승인비율']]

cate_test = test[['CODE_GENDER','CNT_CHILDREN','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE']]
conti_test = test[['AMT_INCOME_TOTAL','age','고용개월','MONTHS_BALANCE','begin','승인비율']]

y_train=train[['STATUS']]
y_test=test[['STATUS']]

In [9]:
cate_train_dummy=pd.get_dummies(cate_train)
cate_test_dummy=pd.get_dummies(cate_test)

scaler_train=StandardScaler()
conti_train_st=scaler_train.fit_transform(conti_train)
scaler_test=StandardScaler()
conti_test_st=scaler_test.fit_transform(conti_test)

In [10]:
conti_train_st=pd.DataFrame(conti_train_st, columns=conti_train.columns)
conti_test_st=pd.DataFrame(conti_test_st, columns=conti_test.columns)

cate_train_dummy.reset_index(inplace=True, drop=True)
conti_train_st.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
cate_test_dummy.reset_index(inplace=True, drop=True)
conti_test_st.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [11]:
train_set=pd.concat([conti_train_st,cate_train_dummy, y_train], axis=1)
test_set=pd.concat([conti_test_st,cate_test_dummy, y_test], axis=1)

## K-fold Cross Validation 으로 모델 신뢰도 평가

In [12]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, make_scorer

In [13]:
X = np.array(train_set.drop(['STATUS'], axis=1)) 
y = train_set['STATUS']

# split 개수, 셔플 여부 및 seed 설정
kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [14]:
train_set['STATUS'].value_counts()

0    130224
1      2173
Name: STATUS, dtype: int64

### 1. Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
accuracy_history = []
recall_history = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LogisticRegression(penalty = 'none', solver='saga',random_state=0) 
    model.fit(X_train, y_train) # 모델 학습

    y_pred = model.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) 
    recall_history.append(recall_score(y_pred, y_test, pos_label = 1)) 

print("각 분할의 Accuracy 기록 :", accuracy_history)
print("Accuracy 평균:", np.mean(accuracy_history))
print("Accuracy 표준편차:", np.std(accuracy_history))

print("각 분할의 recall 기록 :", recall_history)
print("recall 평균 :", np.mean(recall_history))
print("recall 표준편차:", np.std(recall_history))

C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep

각 분할의 Accuracy 기록 : [0.9858761329305136, 0.9845921450151057, 0.9837229502624721, 0.9833452924959402, 0.9852335813286001]
Accuracy 평균: 0.9845540204065264
Accuracy 표준편차: 0.0009341240608014769
각 분할의 recall 기록 : [0.593939393939394, 0.6028708133971292, 0.5397727272727273, 0.5174418604651163, 0.5955056179775281]
recall 평균 : 0.5699060826103789
recall 표준편차: 0.034583589893204696


C:\Users\82102\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### 2. Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
accuracy_history = []
recall_history = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestClassifier(random_state=0) # 모델 선언
    model.fit(X_train, y_train) # 모델 학습

    y_pred = model.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) 
    recall_history.append(recall_score(y_pred, y_test, pos_label = 1)) 

print("각 분할의 Accuracy 기록 :", accuracy_history)
print("Accuracy 평균:", np.mean(accuracy_history))
print("Accuracy 표준편차:", np.std(accuracy_history))

print("각 분할의 recall 기록 :", recall_history)
print("recall 평균 :", np.mean(recall_history))
print("recall 표준편차:", np.std(recall_history))

각 분할의 Accuracy 기록 : [0.9853474320241692, 0.9834214501510574, 0.983307526719287, 0.9833452924959402, 0.9845915631254957]
Accuracy 평균: 0.9840026529031899
Accuracy 표준편차: 0.0008256350149807576
각 분할의 recall 기록 : [0.5216284987277354, 0.5148514851485149, 0.5037220843672456, 0.5071090047393365, 0.5231607629427792]
recall 평균 : 0.5140943671851224
recall 표준편차: 0.007693054492194416


### 3.SVM

In [19]:
from sklearn.svm import SVC

In [20]:
accuracy_history = []
recall_history = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = SVC(kernel = 'rbf',random_state=0) # 모델 선언
    model.fit(X_train, y_train) # 모델 학습

    y_pred = model.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) 
    recall_history.append(recall_score(y_pred, y_test, pos_label = 1)) 

print("각 분할의 Accuracy 기록 :", accuracy_history)
print("Accuracy 평균:", np.mean(accuracy_history))
print("Accuracy 표준편차:", np.std(accuracy_history))

print("각 분할의 recall 기록 :", recall_history)
print("recall 평균 :", np.mean(recall_history))
print("recall 표준편차:", np.std(recall_history))

각 분할의 Accuracy 기록 : [0.9862160120845922, 0.9856495468277946, 0.9848559235620681, 0.9847048604554552, 0.9858755995317043]
Accuracy 평균: 0.9854603884923229
Accuracy 표준편차: 0.0005857215315655816
각 분할의 recall 기록 : [0.696078431372549, 0.8034188034188035, 0.7037037037037037, 0.723404255319149, 0.7575757575757576]
recall 평균 : 0.7368361902779925
recall 표준편차: 0.03950379090336953


### 4. XGB

In [21]:
from xgboost import XGBClassifier

In [22]:
accuracy_history = []
recall_history = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = XGBClassifier(random_state=0) # 모델 선언
    model.fit(X_train, y_train) # 모델 학습

    y_pred = model.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) 
    recall_history.append(recall_score(y_pred, y_test, pos_label = 1)) 

print("각 분할의 Accuracy 기록 :", accuracy_history)
print("Accuracy 평균:", np.mean(accuracy_history))
print("Accuracy 표준편차:", np.std(accuracy_history))

print("각 분할의 recall 기록 :", recall_history)
print("recall 평균 :", np.mean(recall_history))
print("recall 표준편차:", np.std(recall_history))

C:\Users\82102\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:51:52] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\82102\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:51:55] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\82102\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:51:59] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\82102\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:52:02] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\82102\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:52:06] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
각 분할의 Accuracy 기록 : [0.9874244712990936, 0.9861027190332327, 0.9854601759885192, 0.9860644284149703, 0.9868197439480343]
Accuracy 평균: 0.9863743077367699
Accuracy 표준편차: 0.0006792171077153273
각 분할의 recall 기록 : [0.6935483870967742, 0.7106598984771574, 0.6666666666666666, 0.7074468085106383, 0.7065217391304348]
recall 평균 : 0.6969686999763344
recall 표준편차: 0.01623830037177331


### 5. KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
accuracy_history = []
recall_history = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = KNeighborsClassifier() # 모델 선언
    model.fit(X_train, y_train) # 모델 학습

    y_pred = model.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) 
    recall_history.append(recall_score(y_pred, y_test, pos_label = 1)) 

print("각 분할의 Accuracy 기록 :", accuracy_history)
print("Accuracy 평균:", np.mean(accuracy_history))
print("Accuracy 표준편차:", np.std(accuracy_history))

print("각 분할의 recall 기록 :", recall_history)
print("recall 평균 :", np.mean(recall_history))
print("recall 표준편차:", np.std(recall_history))

각 분할의 Accuracy 기록 : [0.9868202416918429, 0.9861404833836858, 0.9847426262321085, 0.9851958155519468, 0.9854601759885192]
Accuracy 평균: 0.9856718685696206
Accuracy 표준편차: 0.0007312034215125868
각 분할의 recall 기록 : [0.6147540983606558, 0.6640625, 0.5773584905660377, 0.6131687242798354, 0.5735294117647058]
recall 평균 : 0.608574644994247
recall 표준편차: 0.03268274753116117
